#Aplicação Geral - Exportações e Importações

Tratamento dos arquivos brutos:

lendo os arquivos das tabelas relacionais, limpando , renomeando e excluindo duplicações

In [ ]:
# importando biblioteca
import pandas as pd

mun = 'https://balanca.economia.gov.br/balanca/bd/tabelas/UF_MUN.csv'
sh4 = 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_SH.csv'
pais = 'https://balanca.economia.gov.br/balanca/bd/tabelas/PAIS.csv'

df_mun = pd.read_csv(mun, sep=";", encoding="latin1")
df_sh4 = pd.read_csv(sh4, sep=";", encoding="latin1",usecols=['CO_SH4','NO_SH4_POR'])
df_pais = pd.read_csv(pais, sep=";", encoding="latin1",usecols=['CO_PAIS', 'CO_PAIS_ISON3', 'CO_PAIS_ISOA3', 'NO_PAIS'])

# renomeando colunas
df_mun = df_mun.rename(columns={"CO_MUN_GEO": "CO_MUN"})
df_sh4 = df_sh4.rename(columns={"NO_SH4_POR": "PRODUTO",'CO_SH4':'SH4'})

# excluindo duplicação e tirando o index
df_mun = df_mun.drop_duplicates().reset_index(drop=True)
df_sh4 = df_sh4.drop_duplicates().reset_index(drop=True)
df_pais = df_pais.drop_duplicates().reset_index(drop=True)


**Exportação**

criando a pasta de exportação;
lendo os dados dos anos de 1997 até 2025 da tabela principal;
realizando os filtros iniciais e mesclando as tabelas(principal com as tabelas relacionais);
criação da coluna valor_agregado e da coluna ano_mes; tirando acento das palavras(coluna produto)

In [ ]:
# importando bibliotecas
import os
import unicodedata

# criando a pasta para salvar os dfs
pasta = 'exportação'
if not os.path.exists(pasta):
  os.makedirs(pasta)


def remove_acento(texto: str) -> str:
  """
    Remove acentos de uma string.

    Args:
        texto: A string com acentos.

    Returns:
        A string sem acentos.
    """
  texto_sem_acentos = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
  return texto_sem_acentos



for ano in range(1997, 2026):
  url = f'https://balanca.economia.gov.br/balanca/bd/comexstat-bd/mun/EXP_{ano}_MUN.csv'
  df = pd.read_csv(url, sep=";", encoding="latin1", chunksize=100000)

  try:
    for chunk in df:
      filtro = chunk[
          (chunk['KG_LIQUIDO'] > 0) &
          (chunk['VL_FOB'] > 0) &
          (chunk['CO_PAIS'] != 0)
      ]

      filtro = filtro.merge(df_mun[['CO_MUN','NO_MUN']], on='CO_MUN', how='left')
      filtro = filtro.merge(df_sh4[['SH4', 'PRODUTO']], on='SH4', how='left')
      filtro = filtro.merge(df_pais[['CO_PAIS','NO_PAIS','CO_PAIS_ISOA3']], on='CO_PAIS', how='left')

      filtro['VALOR_AGREGADO'] = filtro['VL_FOB'] / filtro['KG_LIQUIDO']
      filtro['ANO_MES'] = filtro['CO_ANO'].astype(str) + '-' + filtro['CO_MES'].astype(str).str.zfill(2)
      # filtrando para não pegar o que tiver cod do pai igual zzz
      filtro = filtro[filtro['CO_PAIS_ISOA3'] != 'ZZZ']
      filtro = filtro[filtro['SG_UF_MUN'] != 'EX']
      # tirando os acentos das palavras da coluna produo
      filtro['PRODUTO'] = filtro['PRODUTO'].apply(remove_acento)

      # Especificando a nova ordem das colunas
      nova_ordem = ['ANO_MES', 'NO_MUN', 'CO_MUN','SG_UF_MUN','NO_PAIS','PRODUTO','SH4','CO_PAIS','KG_LIQUIDO','VL_FOB','VALOR_AGREGADO']

      # Reorganizando as colunas
      filtro = filtro[nova_ordem]

      filtro.to_parquet(f'/content/{pasta}/EXP_{ano}_MUN.parquet', index=False)

  except Exception as e:
    print(f"Erro ao processar ano {ano}: {e}")

junta todos os dados, de todos os anos em um dataframe só;
deixa a letra do prodto maiúscula; pode baixar o aquivo csv

In [ ]:
import glob

filtrado = glob.glob('/content/exportação/EXP_*_MUN.parquet')

result_filtrado_exp = []

for resultado in filtrado:
  df_filtrado = pd.read_parquet(resultado)
  result_filtrado_exp.append(df_filtrado)

result_exp = pd.concat(result_filtrado_exp)
result_exp['PRODUTO'] = result_exp['PRODUTO'].str.upper()


# caso queira baixar o arquivo csv de todos os anos e municipios de exportação, tirar o comentário(#) abaixo
# result_exp.to_csv('result_df_geral_exp.csv', index=False)
result_exp

,ANO_MES,NO_MUN,CO_MUN,SG_UF_MUN,NO_PAIS,PRODUTO,SH4,CO_PAIS,KG_LIQUIDO,VL_FOB,VALOR_AGREGADO
0,2009-09,CHUI,4305439,RS,Uruguai,SERVICOS DE MESA E OUTROS ARTIGOS DE USO DOMES...,3924,845,656,3104,4.731707
1,2009-09,BARUERI,3405708,SP,México,"TUBOS DE BORRACHA VULCANIZADA NAO ENDURECIDA, ...",4009,493,168,5316,31.642857
2,2009-09,SAO BERNARDO DO CAMPO,3448708,SP,Argentina,TAPETES E OUTROS REVESTIMENTOS PARA PAVIMENTOS...,5704,63,5,36,7.200000
3,2009-09,FRANCA,3416200,SP,Angola,"VESTUARIO E SEUS ACESSORIOS, DE COURO NATURAL ...",4203,40,9,233,25.888889
4,2009-09,CARPINA,2604007,PE,Uruguai,OUTROS MOVEIS E SUAS PARTES,9403,845,371,1510,4.070081
...,...,...,...,...,...,...,...,...,...,...,...
27268,2012-04,BARUERI,3405708,SP,Bolívia,"OUTRAS CHAPAS, FOLHAS, PELICULAS, TIRAS E LAMI...",3920,97,84915,210849,2.483060
27269,2012-04,SOBRAL,2312908,CE,África do Sul,OUTRO CALCADO COM SOLA EXTERIOR E PARTE SUPERI...,6402,756,510,16038,31.447059
27270,2012-04,PINDAMONHANGABA,3438006,SP,México,"REFRIGERADORES, CONGELADORES (FREEZERS) E OUTR...",8418,493,4017,22214,5.529998
27271,2012-04,SUZANO,3452502,SP,Índia,"ROLAMENTOS DE ESFERAS, DE ROLETES OU DE AGULHAS",8482,361,18,1661,92.277778


criação dos dropdowns para serem selecionados e criar o dataframe filtrado de forma personalizada; criação do botão de filtrar e de gerar csv; dataframe e arquivo .csv com filtros personalizados;

In [ ]:
import ipywidgets as widgets
from IPython.display import display,clear_output

# para selecionar o estado(pela sigla)
estado = widgets.Dropdown(
    options = sorted(result_exp['SG_UF_MUN'].unique()),
    description = 'Estado'
)

# para selecionar data inicial
data_inicial = widgets.Dropdown(
    options = sorted(result_exp['ANO_MES'].unique()),
    description = 'Data Inicial'
)

# para selecionar data final
data_final = widgets.Dropdown(
    options = sorted(result_exp['ANO_MES'].unique()),
    description = 'Data Final'
)

btn_csv = widgets.Button(
    description = 'Exportar csv'
)

output = widgets.Output()

# para selecionar os municipio
# ele mostra os municipios de acordo com o estado que foi selecionado anteriormente, a função mais abaixo
container_mun = widgets.VBox([])
btn_mais_mun = widgets.Button(description='Adicionar município')
contador_mun = widgets.IntText(value=0, layout=widgets.Layout(display='none'))

qtd_max_mun = 10

filtro = pd.DataFrame()
filtro = None

# função que atuaiza a lista de municipio de acordo com o estado
def municipio_atualizado(*args):
  mun = result_exp[result_exp['SG_UF_MUN'] == estado.value]['NO_MUN'].unique()
  for dropdown in container_mun.children:
        dropdown.options = sorted(mun)


# Adiciona novo dropdown de município
def adicionar_mun(btn):
    if contador_mun.value < qtd_max_mun:
        mun_options = sorted(result_exp[result_exp['SG_UF_MUN'] == estado.value]['NO_MUN'].unique())
        municipio = widgets.Dropdown(
            options=mun_options,
            description=f'Município {contador_mun.value + 1}'
        )
        container_mun.children += (municipio,)
        contador_mun.value += 1
    else:
        btn_mais_mun.description = 'Limite atingido'
        btn_mais_mun.disabled = True

btn_mais_mun.on_click(adicionar_mun)
estado.observe(municipio_atualizado, names='value')

# Filtra o DataFrame com base nas seleções
def aplicar_filtros(btn):
    global filtro
    with output:
        clear_output()
        municipios_selecionados = [w.value for w in container_mun.children]
        if not municipios_selecionados:
          filtro = result_exp[
            (result_exp['SG_UF_MUN'] == estado.value) &
            (result_exp['ANO_MES'] >= data_inicial.value) &
            (result_exp['ANO_MES'] <= data_final.value)
          ]
        else:
          filtro = result_exp[
              (result_exp['SG_UF_MUN'] == estado.value) &
              (result_exp['ANO_MES'] >= data_inicial.value) &
              (result_exp['ANO_MES'] <= data_final.value) &
              (result_exp['NO_MUN'].isin(municipios_selecionados))
          ]
        display(filtro)


def exportar_csv(btn):
  global filtro
  if filtro is not None and not filtro.empty:
    junta_nome_mun = ''
    for n_m in container_mun.children:
      nome_mun = n_m.value
      mun_nome = nome_mun.replace(' ','-')
      junta_nome_mun = junta_nome_mun + '_' + mun_nome
    data = data_inicial.value + '_' + data_final.value
    nome_aquivo = f'df_exp{junta_nome_mun}_{data}.csv'
    filtro.to_csv(f'{nome_aquivo}', index=False)
    with output:
      print(f'Arquivo csv {nome_aquivo} exportado com sucesso')
  else:
    with output:
      print('Nenhum dado para exportar')

btn_csv.on_click(exportar_csv)

btn_filtrar = widgets.Button(description='Filtrar dados')
btn_filtrar.on_click(aplicar_filtros)

# Layout final
display(estado, data_inicial, data_final, btn_mais_mun, container_mun, btn_filtrar, output,btn_csv)

Dropdown(description='Estado', options=('AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS'…

Dropdown(description='Data Inicial', options=('1997-01', '1997-02', '1997-03', '1997-04', '1997-05', '1997-06'…

Dropdown(description='Data Final', options=('1997-01', '1997-02', '1997-03', '1997-04', '1997-05', '1997-06', …

Button(description='Adicionar município', style=ButtonStyle())

VBox()

Button(description='Filtrar dados', style=ButtonStyle())

Output()

Button(description='Exportar csv', style=ButtonStyle())

**Importação**

criando a pasta de importação; lendo os dados dos anos de 1997 até 2025 da tabela principal; realizando os filtros iniciais e mesclando as tabelas(principal com as tabelas relacionais); criação da coluna valor_agregado e da coluna ano_mes; tirando acento das palavras(coluna produto)

In [ ]:
import pandas as pd
import unicodedata

# importando biblioteca
import os

# criando a pasta para salvar os dfs
pasta = 'importação'
if not os.path.exists(pasta):
  os.makedirs(pasta)

def remove_acento(texto: str) -> str:
  """
    Remove acentos de uma string.

    Args:
        texto: A string com acentos.

    Returns:
        A string sem acentos.
    """
  texto_sem_acentos = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
  return texto_sem_acentos


for ano in range(1997, 2026):
  url = f'https://balanca.economia.gov.br/balanca/bd/comexstat-bd/mun/IMP_{ano}_MUN.csv'
  df = pd.read_csv(url, sep=";", encoding="latin1", chunksize=100000)

  try:
    for chunk in df:
      filtro = chunk[
          (chunk['KG_LIQUIDO'] > 0) &
          (chunk['VL_FOB'] > 0) &
          (chunk['CO_PAIS'] != 0)
      ]

      filtro = filtro.merge(df_mun[['CO_MUN','NO_MUN_MIN','NO_MUN']].drop_duplicates(), on='CO_MUN', how='left')
      filtro = filtro.merge(df_sh4[['SH4', 'PRODUTO']].drop_duplicates(), on='SH4', how='left')
      filtro = filtro.merge(df_pais[['CO_PAIS','NO_PAIS','CO_PAIS_ISOA3']].drop_duplicates(), on='CO_PAIS', how='left')

      filtro['VALOR_AGREGADO'] = filtro['VL_FOB'] / filtro['KG_LIQUIDO']
      filtro['ANO_MES'] = filtro['CO_ANO'].astype(str) + '-' + filtro['CO_MES'].astype(str).str.zfill(2)

      # filtrando para não pegar o que tiver cod do pai igual zzz
      filtro = filtro[filtro['CO_PAIS_ISOA3'] != 'ZZZ']
      filtro = filtro[filtro['SG_UF_MUN'] != 'EX']

      # tirando os acentos das palavras da coluna produo
      filtro['PRODUTO'] = filtro['PRODUTO'].apply(remove_acento)

      # Especificando a nova ordem das colunas
      nova_ordem = ['ANO_MES', 'NO_MUN', 'CO_MUN','SG_UF_MUN','NO_PAIS','PRODUTO','SH4','CO_PAIS','KG_LIQUIDO','VL_FOB','VALOR_AGREGADO']

      # Reorganizando as colunas
      filtro = filtro[nova_ordem]

      filtro.to_parquet(f'/content/{pasta}/IMP_{ano}_MUN.parquet', index=False)

  except Exception as e:
    print(f"Erro ao processar ano {ano}: {e}")


junta todos os dados, de todos os anos em um dataframe só; deixa a letra do prodto maiúscula; pode baixar o aquivo csv

In [ ]:
import glob

filtrado = glob.glob('/content/importação/IMP_*_MUN.parquet')

result_filtrado_imp = []

for resultado in filtrado:
  df_filtrado = pd.read_parquet(resultado)
  result_filtrado_imp.append(df_filtrado)

result_imp = pd.concat(result_filtrado_imp)
result_imp['PRODUTO'] = result_imp['PRODUTO'].str.upper()

# caso queira baixar o arquivo csv de todos os anos e municipios de exportação, tirar o comentário(#) abaixo
# result_imp.to_csv('result_filtrado_imp.csv', index=False)

result_imp

,ANO_MES,NO_MUN,CO_MUN,SG_UF_MUN,NO_PAIS,PRODUTO,SH4,CO_PAIS,KG_LIQUIDO,VL_FOB,VALOR_AGREGADO
0,2007-08,SAO PAULO,3450308,SP,Taiwan (Formosa),"TRANSFORMADORES ELETRICOS, CONVERSORES ELETRIC...",8504,161,13318,158987,11.937753
1,2007-01,SANTOS,3448500,SP,Coreia do Sul,"CENTRIFUGADORES, INCLUIDOS OS SECADORES CENTRI...",8421,190,49,121,2.469388
2,2007-06,JUNDIAI,3425904,SP,Estados Unidos,SUPORTES PREPARADOS PARA GRAVACAO DE SOM OU PA...,8523,249,1893,3403,1.797676
3,2007-06,SAO PAULO,3450308,SP,Estados Unidos,"CONDENSADORES ELETRICOS, FIXOS, VARIAVEIS OU A...",8532,249,835,71510,85.640719
4,2007-07,ITU,3423909,SP,Alemanha,PERFIS DE FERRO OU ACO NAO LIGADO,7216,23,2978,9883,3.318670
...,...,...,...,...,...,...,...,...,...,...,...
83080,1998-10,BRASILIA,5400108,DF,Estados Unidos,"ACUMULADORES ELETRICOS E SEUS SEPARADORES, MES...",8507,249,50,567,11.340000
83081,1998-01,PONTA GROSSA,4119905,PR,Reino Unido,"RELOGIOS DE PULSO, RELOGIOS DE BOLSO E RELOGIO...",9102,628,50,2200,44.000000
83082,1998-01,BARUERI,3405708,SP,Itália,"LIVROS DE REGISTO E DE CONTABILIDADE, BLOCOS D...",4820,386,22,5589,254.045455
83083,1998-01,CARAPICUIBA,3410609,SP,Espanha,"TECIDOS DE ALGODAO, CONTENDO PELO MENOS 85 %, ...",5209,245,16,535,33.437500


criação dos dropdowns para serem selecionados e criar o dataframe filtrado de forma personalizada; criação do botão de filtrar e de gerar csv; dataframe e arquivo .csv com filtros personalizados;

In [ ]:
import ipywidgets as widgets
from IPython.display import display,clear_output

# para selecionar o estado(pela sigla)
estado = widgets.Dropdown(
    options = sorted(result_imp['SG_UF_MUN'].unique()),
    description = 'Estado'
)

# para selecionar data inicial
data_inicial = widgets.Dropdown(
    options = sorted(result_imp['ANO_MES'].unique()),
    description = 'Data Inicial'
)

# para selecionar data final
data_final = widgets.Dropdown(
    options = sorted(result_imp['ANO_MES'].unique()),
    description = 'Data Final'
)

btn_csv_imp = widgets.Button(
    description = 'Exportar csv'
)

output = widgets.Output()

# para selecionar os municipio
# ele mostra os municipios de acordo com o estado que foi selecionado anteriormente, a função mais abaixo
container_mun = widgets.VBox([])
btn_mais_mun = widgets.Button(description='Adicionar município')
contador_mun = widgets.IntText(value=0, layout=widgets.Layout(display='none'))

qtd_max_mun = 10

filtro = pd.DataFrame()
filtro = None

# função que atuaiza a lista de municipio de acordo com o estado
def municipio_atualizado(*args):
  mun = result_imp[result_imp['SG_UF_MUN'] == estado.value]['NO_MUN'].unique()
  for dropdown in container_mun.children:
        dropdown.options = sorted(mun)


# Adiciona novo dropdown de município
def adicionar_mun(btn):
    if contador_mun.value < qtd_max_mun:
        mun_options = sorted(result_imp[result_imp['SG_UF_MUN'] == estado.value]['NO_MUN'].unique())
        municipio = widgets.Dropdown(
            options=mun_options,
            description=f'Município {contador_mun.value + 1}'
        )
        container_mun.children += (municipio,)
        contador_mun.value += 1
    else:
        btn_mais_mun.description = 'Limite atingido'
        btn_mais_mun.disabled = True

btn_mais_mun.on_click(adicionar_mun)
estado.observe(municipio_atualizado, names='value')


# Filtra o DataFrame com base nas seleções
def aplicar_filtros(btn):
    global filtro
    with output:
        clear_output()
        municipios_selecionados = [w.value for w in container_mun.children]
        if not municipios_selecionados:
          filtro = result_imp[
            (result_imp['SG_UF_MUN'] == estado.value) &
            (result_imp['ANO_MES'] >= data_inicial.value) &
            (result_imp['ANO_MES'] <= data_final.value)
          ]
        else:
          filtro = result_imp[
            (result_imp['SG_UF_MUN'] == estado.value) &
            (result_imp['ANO_MES'] >= data_inicial.value) &
            (result_imp['ANO_MES'] <= data_final.value) &
            (result_imp['NO_MUN'].isin(municipios_selecionados))
          ]
        display(filtro)

def exportar_csv_imp(btn):
  global filtro
  if filtro is not None and not filtro.empty:
    junta_nome_mun = ''
    for n_m in container_mun.children:
      nome_mun = n_m.value
      mun_nome = nome_mun.replace(' ','-')
      junta_nome_mun = junta_nome_mun + '_' + mun_nome
    data = data_inicial.value + '_' + data_final.value
    nome_aquivo = f'df_imp{junta_nome_mun}_{data}.csv'
    filtro.to_csv(f'{nome_aquivo}', index=False)
    with output:
      print(f'Arquivo csv {nome_aquivo} exportado com sucesso')
  else:
    with output:
      print('Nenhum dado para exportar')
    print(f"filtro está vazio: {filtro}")

btn_csv_imp.on_click(exportar_csv_imp)

btn_filtrar = widgets.Button(description='Filtrar dados')
btn_filtrar.on_click(aplicar_filtros)

# Layout final
display(estado, data_inicial, data_final, btn_mais_mun, container_mun, btn_filtrar, output,btn_csv_imp)

Dropdown(description='Estado', options=('AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS'…

Dropdown(description='Data Inicial', options=('1997-01', '1997-02', '1997-03', '1997-04', '1997-05', '1997-06'…

Dropdown(description='Data Final', options=('1997-01', '1997-02', '1997-03', '1997-04', '1997-05', '1997-06', …

Button(description='Adicionar município', style=ButtonStyle())

VBox()

Button(description='Filtrar dados', style=ButtonStyle())

Output()

Button(description='Exportar csv', style=ButtonStyle())